In [8]:
import requests, json
import numpy as np

def getBestTimeStamps(videoId: str, cutoff: float = 0.8):
    url = f'https://yt.lemnoslife.com/videos?part=mostReplayed&id={videoId}'
    content = requests.get(url).text
    data = json.loads(content)

    datapoints = np.empty((100,))
    t_step = data['items'][0]['mostReplayed']['heatMarkers'][0]['heatMarkerRenderer']['markerDurationMillis']
    for i, heatMarker in enumerate(data['items'][0]['mostReplayed']['heatMarkers']):
        heatMarker = heatMarker['heatMarkerRenderer']
        intensityScoreNormalized = heatMarker['heatMarkerIntensityScoreNormalized']
        datapoints[i] = intensityScoreNormalized

    new_start = np.argmax((datapoints[1:] - datapoints[:-1]) > 0)
    datapoints[:new_start] = 0
    datapoints *= 1 / np.max(datapoints)

    peak = np.argmax(datapoints)

    good_dp = datapoints > cutoff
    prev_dp = ~good_dp[:peak][::-1]
    if prev_dp.any():
        start_idx = peak - np.argmax(prev_dp)
    else:
        start_idx = 0

    post_dp = ~good_dp[peak:]
    if post_dp.any():
        end_idx = peak + np.argmax(post_dp)
    else:
        end_idx = good_dp.shape[0]

    start_ms = start_idx * t_step
    end_ms = end_idx * t_step

    return (start_ms, end_ms)

def display_ms(ms):
    total_sec = ms // 1000
    mts = total_sec // 60
    sec = total_sec % 60
    print('%d:%.2d' % (mts, sec))

In [9]:
start_ms, end_ms = getBestTimeStamps('q9X9g2t67PM')
display_ms(start_ms)
display_ms(end_ms)
print(start_ms)
print(end_ms)

1480
1:08
2:28
68080
148000
